In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy.stats import percentileofscore as score


In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secret import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
# api_url = f'https://sandbox.iexapis.com/stable/time-series/fundamentals/{symbol}?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2576011950748,
 'week52high': 161.09,
 'week52low': 115.1,
 'week52highSplitAdjustOnly': 158.63,
 'week52lowSplitAdjustOnly': 113.61,
 'week52change': 0.2703766973796451,
 'sharesOutstanding': 16545555560,
 'float': 0,
 'avg10Volume': 58374014,
 'avg30Volume': 69507169,
 'day200MovingAvg': 141.65,
 'day50MovingAvg': 148.51,
 'employees': 148422,
 'ttmEPS': 11.4,
 'ttmDividendRate': 0.8808116705500265,
 'dividendYield': 0.005915813588910392,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-29',
 'nextEarningsDate': '2022-01-16',
 'peRatio': 13.196361420694084,
 'beta': 1.4050932363309394,
 'maxChangePercent': 58.24064529027505,
 'year5ChangePercent': 5.114238746305149,
 'year2ChangePercent': 1.3411086230515707,
 'year1ChangePercent': 0.2752304798545658,
 'ytdChangePercent': 0.1422427790171327,
 'month6ChangePercent': 0.2095219674078944,
 'month3ChangePercent': 0.007538417704749118,
 'month1ChangePercent': 0.0672687513954817,
 'day30ChangePe

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
# api_url = f'https://sandbox.iexapis.com/stable/time-series/fundamentals/{symbol}?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2514300014803,
 'week52high': 164.35,
 'week52low': 117.1,
 'week52highSplitAdjustOnly': 164.04,
 'week52lowSplitAdjustOnly': 116.69,
 'week52change': 0.2716912791294674,
 'sharesOutstanding': 16854317486,
 'float': 0,
 'avg10Volume': 57898446,
 'avg30Volume': 70216918,
 'day200MovingAvg': 146.64,
 'day50MovingAvg': 148.49,
 'employees': 148839,
 'ttmEPS': 11.26,
 'ttmDividendRate': 0.8951769033866357,
 'dividendYield': 0.005866644473193749,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-31',
 'nextEarningsDate': '2022-01-22',
 'peRatio': 13.271356901905422,
 'beta': 1.4622163131313601,
 'maxChangePercent': 58.38648139068884,
 'year5ChangePercent': 4.921558083264326,
 'year2ChangePercent': 1.3069600748945889,
 'year1ChangePercent': 0.2745451265977161,
 'ytdChangePercent': 0.14263745788420104,
 'month6ChangePercent': 0.20721894695993912,
 'month3ChangePercent': 0.00764277914076384,
 'month1ChangePercent': 0.06732392811524249,
 'day30Chang

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# for symbol_string in symbol_strings:
#     print(symbol_string)


my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()
    # print(symbol_string.split(','))
#     print(data['AAPL']['stats'])
    for symbol in symbol_string.split(','): 
        final_dataframe= final_dataframe.append(
        
            pd.Series(
                [
                 symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    
                    'N/A'
                ],index = my_columns
            ), ignore_index = True)
final_dataframe
#     print(data.status_code)
# print(data)

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,165.23,0.476502,N/A
1,AAL,20.80,0.668857,N/A
2,AAP,248.01,0.590317,N/A
3,AAPL,155.77,0.274960,N/A
4,ABBV,117.66,0.247158,N/A
...,...,...,...,...
95,CINF,125.88,0.614245,N/A
96,CL,81.23,-0.067057,N/A
97,CLX,171.70,-0.182997,N/A
98,CMA,89.76,0.871559,N/A


In [7]:
final_dataframe.sort_values('One-Year Price Return',ascending=False, inplace =True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,23,ALB,279.90,1.330073,N/A
1,29,AMAT,162.73,1.218189,N/A
2,89,CF,66.07,1.196520,N/A
3,19,AIV,8.37,0.958367,N/A
4,37,ANET,534.70,0.953743,N/A
5,98,CMA,89.76,0.871559,N/A
6,82,CBRE,105.78,0.858850,N/A
7,31,AMD,149.21,0.842510,N/A
8,60,BEN,37.38,0.825611,N/A
9,56,BAC,49.16,0.803825,N/A


In [8]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]
hqm_dataframe = pd.DataFrame(columns = hqm_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'] ,
                'N/A',

                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ], index = hqm_columns
            ),ignore_index = True
        )
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,163.01,N/A,0.473831,N/A,0.242143,N/A,0.012156,N/A,0.07442,N/A,N/A
1,AAL,20.72,N/A,0.664911,N/A,-0.045447,N/A,0.023624,N/A,0.0349,N/A,N/A
2,AAP,246.95,N/A,0.596697,N/A,0.188695,N/A,0.165038,N/A,0.137146,N/A,N/A
3,AAPL,150.66,N/A,0.274388,N/A,0.21152,N/A,0.007788,N/A,0.066641,N/A,N/A
4,ABBV,120.59,N/A,0.241203,N/A,0.027611,N/A,0.016891,N/A,0.09226,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.20,N/A,0.243356,N/A,0.070509,N/A,-0.069521,N/A,0.013988,N/A,N/A
501,ZBH,135.08,N/A,-0.123042,N/A,-0.198797,N/A,-0.0963,N/A,-0.075132,N/A,N/A
502,ZBRA,609.60,N/A,0.74254,N/A,0.258581,N/A,0.037587,N/A,0.19264,N/A,N/A
503,ZION,65.60,N/A,0.802179,N/A,0.125067,N/A,0.190645,N/A,0.062484,N/A,N/A


In [9]:

hqm_dataframe = hqm_dataframe[hqm_dataframe['One-Year Price Return'].isnull().values ==False]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,163.01,N/A,0.473831,N/A,0.242143,N/A,0.012156,N/A,0.07442,N/A,N/A
1,AAL,20.72,N/A,0.664911,N/A,-0.045447,N/A,0.023624,N/A,0.0349,N/A,N/A
2,AAP,246.95,N/A,0.596697,N/A,0.188695,N/A,0.165038,N/A,0.137146,N/A,N/A
3,AAPL,150.66,N/A,0.274388,N/A,0.21152,N/A,0.007788,N/A,0.066641,N/A,N/A
4,ABBV,120.59,N/A,0.241203,N/A,0.027611,N/A,0.016891,N/A,0.09226,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,126.20,N/A,0.243356,N/A,0.070509,N/A,-0.069521,N/A,0.013988,N/A,N/A
497,ZBH,135.08,N/A,-0.123042,N/A,-0.198797,N/A,-0.0963,N/A,-0.075132,N/A,N/A
498,ZBRA,609.60,N/A,0.74254,N/A,0.258581,N/A,0.037587,N/A,0.19264,N/A,N/A
499,ZION,65.60,N/A,0.802179,N/A,0.125067,N/A,0.190645,N/A,0.062484,N/A,N/A


In [10]:
time_periods= [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month']

for row in hqm_dataframe.index:
    for time_period in time_periods:
#         hqm_dataframe.loc[row,f'{time_period} Return Percentile']= 0
        hqm_dataframe.loc[row,f'{time_period} Return Percentile'] = score(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])

hqm_dataframe

C:\Users\SweeTat\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,163.01,N/A,0.473831,68.862275,0.242143,83.033932,0.012156,44.111776,0.07442,60.678643,N/A
1,AAL,20.72,N/A,0.664911,83.433134,-0.045447,21.157685,0.023624,47.50499,0.0349,38.522954,N/A
2,AAP,246.95,N/A,0.596697,79.041916,0.188695,73.652695,0.165038,88.423154,0.137146,85.828343,N/A
3,AAPL,150.66,N/A,0.274388,44.91018,0.21152,78.443114,0.007788,41.916168,0.066641,56.686627,N/A
4,ABBV,120.59,N/A,0.241203,41.317365,0.027611,39.520958,0.016891,45.708583,0.09226,69.660679,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,126.20,N/A,0.243356,42.115768,0.070509,50.499002,-0.069521,12.57485,0.013988,27.744511,N/A
497,ZBH,135.08,N/A,-0.123042,3.193613,-0.198797,3.193613,-0.0963,8.782435,-0.075132,3.592814,N/A
498,ZBRA,609.60,N/A,0.74254,88.023952,0.258581,85.628743,0.037587,52.49501,0.19264,94.810379,N/A
499,ZION,65.60,N/A,0.802179,90.219561,0.125067,61.277445,0.190645,92.41517,0.062484,55.08982,N/A


In [11]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

C:\Users\SweeTat\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,163.01,N/A,0.473831,68.862275,0.242143,83.033932,0.012156,44.111776,0.07442,60.678643,64.171657
1,AAL,20.72,N/A,0.664911,83.433134,-0.045447,21.157685,0.023624,47.50499,0.0349,38.522954,47.654691
2,AAP,246.95,N/A,0.596697,79.041916,0.188695,73.652695,0.165038,88.423154,0.137146,85.828343,81.736527
3,AAPL,150.66,N/A,0.274388,44.91018,0.21152,78.443114,0.007788,41.916168,0.066641,56.686627,55.489022
4,ABBV,120.59,N/A,0.241203,41.317365,0.027611,39.520958,0.016891,45.708583,0.09226,69.660679,49.051896
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,126.20,N/A,0.243356,42.115768,0.070509,50.499002,-0.069521,12.57485,0.013988,27.744511,33.233533
497,ZBH,135.08,N/A,-0.123042,3.193613,-0.198797,3.193613,-0.0963,8.782435,-0.075132,3.592814,4.690619
498,ZBRA,609.60,N/A,0.74254,88.023952,0.258581,85.628743,0.037587,52.49501,0.19264,94.810379,80.239521
499,ZION,65.60,N/A,0.802179,90.219561,0.125067,61.277445,0.190645,92.41517,0.062484,55.08982,74.750499


# HQM_Dataframe

In [12]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe.iloc[:50]
hqm_dataframe.reset_index(drop=True, inplace= True )
hqm_dataframe

<ipython-input-12-2000e54d08a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,318.70,N/A,1.342341,98.403194,1.226384,99.800399,0.510104,99.401198,0.47201,100.0,99.401198
1,MCHP,84.29,N/A,1.685358,98.802395,1.360028,100.0,1.230203,100.0,0.188067,94.011976,98.203593
2,F,20.10,N/A,1.29513,97.60479,0.715472,98.203593,0.449728,98.602794,0.264982,97.804391,98.053892
3,ANET,540.30,N/A,0.954044,94.610778,0.684754,98.003992,0.459172,98.802395,0.370031,99.600798,97.754491
4,AMD,148.79,N/A,0.852862,92.41517,1.024144,99.401198,0.344564,97.40519,0.365567,99.401198,97.155689
5,SPG,171.10,N/A,1.327775,98.203593,0.43603,95.608782,0.258123,95.808383,0.204962,95.808383,96.357285
6,ALB,278.20,N/A,1.309299,98.003992,0.77016,98.802395,0.184032,91.217565,0.247438,97.205589,96.307385
7,DXCM,658.02,N/A,0.945694,94.211577,1.04138,99.600798,0.319917,97.005988,0.186074,93.612774,96.107784
8,XLNX,223.90,N/A,0.684917,84.431138,0.897676,99.201597,0.464829,99.001996,0.34069,99.001996,95.409182
9,COTY,10.86,N/A,1.276429,97.40519,0.255805,85.229541,0.319802,96.806387,0.434146,99.800399,94.810379


In [13]:
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data= requests.get(api_url).json()
print(data)


{'avgTotalVolume': 1470373, 'calculationPrice': 'close', 'change': 1.21, 'changePercent': 0.00539, 'close': 0, 'closeSource': 'ficifloa', 'closeTime': None, 'companyName': 'Zoetis Inc - Class A', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': 'p  yduedite5cn1raeil em', 'highTime': 1667098007278, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 220.9, 'iexCloseTime': 1691185689951, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 220.42, 'iexOpenTime': 1655973749053, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1696578035674, 'latestPrice': 221.34, 'latestSource': 'Close', 'latestTime': 'November 12, 2021', 'latestUpdate': 1686637184101, 'latestVolume': None, 'low': 0, 'lowSource': 'erim1aypdei teu 5lcnde ', 'lowTime': 1710390163469, 'marketC

In [14]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])
# >>> strings = ['do', 're', 'mi']
# >>> ','.join(strings)
# 'do,re,mi'
my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [15]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url =f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data)
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
                
            ], index = my_columns
            
            ),ignore_index=True
            
        )
        
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,168.65,52.19,N/A
1,AAL,21.23,-4.18,N/A
2,AAP,248.21,26.37,N/A
3,AAPL,155.23,13.39,N/A
4,ABBV,118.01,28.09,N/A
...,...,...,...,...
500,YUM,128.73,25.01,N/A
501,ZBH,135.48,34.09,N/A
502,ZBRA,620.34,38.71,N/A
503,ZION,68.61,6.33,N/A


In [16]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis = 1, inplace=True)
# final_dataframe.drop('level_0', axis = 1, inplace=True)

In [17]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,761.13,3.55,N/A
1,NRG,36.90,3.64,N/A
2,EBAY,74.85,4.21,N/A
3,MPC,68.30,4.6,N/A
4,LYB,96.99,5.71,N/A
5,COF,164.60,6.12,N/A
6,PRU,110.99,6.31,N/A
7,ZION,68.61,6.33,N/A
8,UNM,27.09,6.85,N/A
9,GS,414.82,6.91,N/A


In [18]:
symbol = 'AAPL'
batch_api_call_url  = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# print(data[symbol]['advanced-stats'])
# data.status_code


# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value / ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']


ev_to_gross_profit = enterprise_value/gross_profit


In [19]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earning Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url= f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
#     print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
#     print(data.status_code)
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        
        try:
            ev_to_ebitda = enterprise_value / ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        
        try:
            ev_to_gross_profit = enterprise_value / gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN       
        
        
        
        
        rv_dataframe = rv_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                 data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A',
            ],
                index = rv_columns),
            ignore_index = True)

In [20]:
len(rv_dataframe[rv_dataframe.isnull().any(axis=1)])


15

In [21]:
for column in ['Price-to-Earning Ratio','Price-to-Book Ratio','Price-to-Sales Ratio', 'EV/EBITDA','EV/GP']:
#     print(column)
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

In [22]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earning Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}
for metric in metrics.keys():
    
    for row in rv_dataframe.index:
#         print(rv_dataframe.loc[row, metric])
        rv_dataframe.loc[row,metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,164.32,N/A,51.29,82.376238,9.88,80.39604,8.2200,82.574257,32.989864,86.534653,15.672112,85.148515,N/A
1,AAL,20.46,N/A,-4.13,5.544554,-1.83,5.544554,0.5373,4.158416,-7.623384,2.376238,1.489805,6.138614,N/A
2,AAP,249.08,N/A,26.34,51.683168,4.68,60.594059,1.3800,18.415842,13.165234,40.990099,3.134272,15.445545,N/A
3,AAPL,156.05,N/A,13.88,24.257426,20.40,93.069307,3.5100,51.089109,10.971335,31.881188,8.530939,58.613861,N/A
4,ABBV,120.32,N/A,28.16,55.940594,15.60,89.70297,3.7700,55.247525,10.675766,29.90099,7.509854,51.089109,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.56,N/A,24.70,47.722772,-4.77,5.148515,5.7000,71.980198,20.869424,67.128713,9.823467,65.742574,N/A
501,ZBH,135.14,N/A,34.83,66.534653,2.26,31.287129,3.6100,52.673267,19.183077,63.168317,5.908705,37.029703,N/A
502,ZBRA,591.97,N/A,38.13,70.891089,11.75,83.762376,5.8300,73.069307,28.071597,79.80198,12.528897,76.237624,N/A
503,ZION,68.64,N/A,6.32,8.118812,1.40,14.257426,2.9400,44.158416,4.938257,5.544554,2.689722,12.277228,N/A


In [23]:
from statistics import mean

for row in rv_dataframe.index:

    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row,'RV Score'] = mean(value_percentiles)
#     print(value_percentiles)
rv_dataframe
    
    

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,164.32,N/A,51.29,82.376238,9.88,80.39604,8.2200,82.574257,32.989864,86.534653,15.672112,85.148515,83.405941
1,AAL,20.46,N/A,-4.13,5.544554,-1.83,5.544554,0.5373,4.158416,-7.623384,2.376238,1.489805,6.138614,4.752475
2,AAP,249.08,N/A,26.34,51.683168,4.68,60.594059,1.3800,18.415842,13.165234,40.990099,3.134272,15.445545,37.425743
3,AAPL,156.05,N/A,13.88,24.257426,20.40,93.069307,3.5100,51.089109,10.971335,31.881188,8.530939,58.613861,51.782178
4,ABBV,120.32,N/A,28.16,55.940594,15.60,89.70297,3.7700,55.247525,10.675766,29.90099,7.509854,51.089109,56.376238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.56,N/A,24.70,47.722772,-4.77,5.148515,5.7000,71.980198,20.869424,67.128713,9.823467,65.742574,51.544554
501,ZBH,135.14,N/A,34.83,66.534653,2.26,31.287129,3.6100,52.673267,19.183077,63.168317,5.908705,37.029703,50.138614
502,ZBRA,591.97,N/A,38.13,70.891089,11.75,83.762376,5.8300,73.069307,28.071597,79.80198,12.528897,76.237624,76.752475
503,ZION,68.64,N/A,6.32,8.118812,1.40,14.257426,2.9400,44.158416,4.938257,5.544554,2.689722,12.277228,16.871287


# RV_Dataframe

In [24]:
rv_dataframe.sort_values('RV Score',ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.21,N/A,7.01,9.108911,0.4961,5.742574,0.4110,2.376238,3.201949,3.168317,0.393903,0.792079,4.237624
1,AAL,20.46,N/A,-4.13,5.544554,-1.8300,5.544554,0.5373,4.158416,-7.623384,2.376238,1.489805,6.138614,4.752475
2,PRU,111.57,N/A,6.30,7.920792,0.6815,6.138614,0.6087,5.148515,4.686480,5.148515,0.593608,1.188119,5.108911
3,AIG,58.71,N/A,9.40,12.871287,0.7850,6.732673,1.0600,12.475248,4.433841,4.356436,0.992430,2.970297,7.881188
4,MET,64.83,N/A,10.84,16.633663,0.8050,7.128713,0.7800,7.524752,5.475933,7.128713,0.773348,1.782178,8.039604
5,ALL,116.43,N/A,10.72,16.237624,1.4000,14.257426,0.6788,5.940594,2.302188,2.970297,0.659115,1.386139,8.158416
6,L,58.77,N/A,9.64,13.465347,0.8190,7.524752,1.0147,11.287129,5.258679,6.336634,0.966589,2.772277,8.277228
7,BA,229.15,N/A,-16.00,4.356436,-9.2000,4.752475,2.1000,32.178218,-35.141763,0.594059,-1412.583577,0.19802,8.415842
8,MCK,231.57,N/A,-7.47,4.950495,-61.3200,2.376238,0.1436,0.792079,8.688087,20.990099,2.870452,13.069307,8.435644
9,AIZ,163.20,N/A,7.54,10.29703,1.6300,18.217822,0.9150,9.70297,1.981209,2.772277,0.889021,2.178218,8.633663


# COMPARISON

In [25]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,318.70,N/A,1.342341,98.403194,1.226384,99.800399,0.510104,99.401198,0.47201,100.0,99.401198
1,MCHP,84.29,N/A,1.685358,98.802395,1.360028,100.0,1.230203,100.0,0.188067,94.011976,98.203593
2,F,20.10,N/A,1.29513,97.60479,0.715472,98.203593,0.449728,98.602794,0.264982,97.804391,98.053892
3,ANET,540.30,N/A,0.954044,94.610778,0.684754,98.003992,0.459172,98.802395,0.370031,99.600798,97.754491
4,AMD,148.79,N/A,0.852862,92.41517,1.024144,99.401198,0.344564,97.40519,0.365567,99.401198,97.155689
5,SPG,171.10,N/A,1.327775,98.203593,0.43603,95.608782,0.258123,95.808383,0.204962,95.808383,96.357285
6,ALB,278.20,N/A,1.309299,98.003992,0.77016,98.802395,0.184032,91.217565,0.247438,97.205589,96.307385
7,DXCM,658.02,N/A,0.945694,94.211577,1.04138,99.600798,0.319917,97.005988,0.186074,93.612774,96.107784
8,XLNX,223.90,N/A,0.684917,84.431138,0.897676,99.201597,0.464829,99.001996,0.34069,99.001996,95.409182
9,COTY,10.86,N/A,1.276429,97.40519,0.255805,85.229541,0.319802,96.806387,0.434146,99.800399,94.810379


In [26]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.21,N/A,7.01,9.108911,0.4961,5.742574,0.4110,2.376238,3.201949,3.168317,0.393903,0.792079,4.237624
1,AAL,20.46,N/A,-4.13,5.544554,-1.8300,5.544554,0.5373,4.158416,-7.623384,2.376238,1.489805,6.138614,4.752475
2,PRU,111.57,N/A,6.30,7.920792,0.6815,6.138614,0.6087,5.148515,4.686480,5.148515,0.593608,1.188119,5.108911
3,AIG,58.71,N/A,9.40,12.871287,0.7850,6.732673,1.0600,12.475248,4.433841,4.356436,0.992430,2.970297,7.881188
4,MET,64.83,N/A,10.84,16.633663,0.8050,7.128713,0.7800,7.524752,5.475933,7.128713,0.773348,1.782178,8.039604
5,ALL,116.43,N/A,10.72,16.237624,1.4000,14.257426,0.6788,5.940594,2.302188,2.970297,0.659115,1.386139,8.158416
6,L,58.77,N/A,9.64,13.465347,0.8190,7.524752,1.0147,11.287129,5.258679,6.336634,0.966589,2.772277,8.277228
7,BA,229.15,N/A,-16.00,4.356436,-9.2000,4.752475,2.1000,32.178218,-35.141763,0.594059,-1412.583577,0.19802,8.415842
8,MCK,231.57,N/A,-7.47,4.950495,-61.3200,2.376238,0.1436,0.792079,8.688087,20.990099,2.870452,13.069307,8.435644
9,AIZ,163.20,N/A,7.54,10.29703,1.6300,18.217822,0.9150,9.70297,1.981209,2.772277,0.889021,2.178218,8.633663


In [27]:
def display_all(df):
    with pd.option_context("display.max_rows", 100000, "display.max_columns", 100000): 
        display(df)

In [28]:
s1 = pd.merge(hqm_dataframe, rv_dataframe, how='inner', on=['Ticker'])

In [29]:
display_all(s1)

,Ticker,Price_x,Number of Shares to Buy_x,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Price_y,Number of Shares to Buy_y,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,APA,30.18,N/A,0.533855,74.850299,0.470649,96.407186,0.650918,99.800399,0.201239,95.209581,91.566866,30.71,N/A,18.72,35.247525,-10.17,4.158416,1.56,22.673267,4.978809,5.940594,2.509811,11.287129,15.861386
1,MRO,16.84,N/A,2.449769,99.800399,0.533974,97.40519,0.395186,98.203593,0.057831,51.896208,86.826347,17.32,N/A,-324.92,0.39604,1.20,11.188119,2.79,41.782178,6.796821,10.693069,4.033406,21.584158,17.128713
2,AZO,1939.47,N/A,0.635467,82.035928,0.233812,81.237525,0.177281,90.01996,0.1217,81.037924,83.582834,1958.37,N/A,10.33,15.247525,-11.00,3.960396,1.38,18.415842,7.058585,11.485149,3.205948,16.039604,13.029703


In [30]:
ark_holding = pd.read_excel('C:/Users/Forge 15 i7/Desktop\LSTM/algorithmic trading/algorithmic-trading-python-master/starter_files/ARK_INNOVATION_ETF_ARKK_HOLDINGS-converted.xlsx')
ark_holding


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Forge 15 i7/Desktop\\LSTM/algorithmic trading/algorithmic-trading-python-master/starter_files/ARK_INNOVATION_ETF_ARKK_HOLDINGS-converted.xlsx'

In [ ]:

ark_holding_1 = ark_holding.rename({'Unnamed: 2': 'Ticker'}, axis=1)
ark_holding_1

In [ ]:
s2 = pd.merge(hqm_dataframe, ark_holding_1, how='inner', on=['Ticker'])
s2

In [ ]:
s3 = pd.merge(rv_dataframe, ark_holding_1, how='inner', on=['Ticker'])
s3